In [1]:
import pandas as pd 
import numpy as np
from math import log2
from Node import * 
from ImpurityMeasure import * 

In [2]:
PATH = '/Users/Sathiesh/MachineLearning/INF283/project_1/csv/mushrooms.csv' #TODO fiks

In [3]:
df = pd.read_csv(PATH)
y = df['class']

In [4]:
X = df.drop(['class'], axis=1)

In [5]:
#Sender inn dataframe til meg 

In [6]:
#Ulempen med å splitte på flere er hvis ikke 

In [7]:
def dataframe_to_numpy(X,y): 
    for column in X.columns: 
        X[column] = pd.factorize(X[column])[0]
    X = X.values #numpy array 
    y = pd.factorize(y)[0]
    return X,y

In [8]:
X,y = dataframe_to_numpy(X,y)

In [9]:
def train_test_split(X,y, size=0.2): 
    #lage en numpy funksjon som tar 
    random_selection = np.random.randint(len(y), size=int(len(y)*size))
    X_test, y_test = X[random_selection],  y[random_selection]
    X_train, y_train = np.delete(X, (random_selection), axis=0), np.delete(y, (random_selection), axis=0)
    return X_train, X_test, y_train, y_test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [11]:
def count_values(y): 
    result = {} 
    for value in y: 
        if value not in result: result[value] = 1
        else: result[value]+=1
    return result

In [12]:
def case_prediction(x,node): #x = [0,0,1,1]
    while len(x) > 0: 
        cat_var = x[node.category] #Rot sin kategori, så da velges posisjon 0 for eksempel 
        x = np.delete(x, node.category, axis=0)
        if cat_var not in node.children: #miss classification  
            return None
        child_node = node.children[cat_var]
        if child_node.isLeaf: 
            return child_node.data
        node = child_node

In [13]:
def predict(x, node):
    result = []
    for i in x: 
        result.append(case_prediction(i,node))
    return np.array(result)

In [14]:
def accuracy(y, pred): 
    result = 0
    for idx, y_ in enumerate(pred): 
        if y_ == y[idx]: result+=1
    return (result/len(y))

In [15]:
def find_best_split(categories, y, impurity_measure): 
    impurity_measure_method = getattr(ImpurityMeasure(), impurity_measure)
    
    impurity_measure_src = impurity_measure_method(y)
    best_ig_score = []
    for cat in categories:
        #KOMMENTAR en cat kan typisk være outlook
        #EN CAT KAN INNEHOLDE VARIABLENE: sunny, overcast, rain 
        variables_dic = {}
        impurity_measure_of_branches = 0 if impurity_measure=='entropy' else impurity_measure_src
        
        #Variabel og y veridene til den 
        for index,var in enumerate(cat): 
            if var not in variables_dic: variables_dic[var] = []
            variables_dic[var].append(y[index])
 
        #Regne ut entropien for hver av variablene og legge dem sammen
        for key, var_y in variables_dic.items(): 
            result = impurity_measure_method(var_y)*(len(var_y)/len(y))
            
            #TODO få dette til å funke ordentlig
            #Regne ut i fra om det er entropy eller gini
            if(impurity_measure == 'entropy'):
                impurity_measure_of_branches += result
            else: 
                 impurity_measure_of_branches -= result
                
        information_gain = impurity_measure_src - impurity_measure_of_branches 
        best_ig_score.append(information_gain)
    #print(best_ig_score)
    return np.argmax(best_ig_score)

In [16]:
import operator #TODO Kan jeg bruke denne? 
def find_dominant_value(values_dic): 
    return max(values_dic, key=values_dic.get)

In [17]:
#definere paramter, type, return verdi
def create_tree(X,y, node, impurity_measure): 
    #Find best category 
    if len(X) == 0: 
        return  #hva skal jeg retunere?????  
    else: 
        index = find_best_split(X,y,impurity_measure) #finner indeksen til den beste kategorien å splitte på
        
        node.category = index
        node.data = X[index] #add the category data to the node 
        child_dic = count_values(node.data) #count num variables for the category 
    
        for i in child_dic.keys(): #Add variable name and Node as child to the Node
            node.add_child(i,Node()) #Verdi[0] variable navn som er gitt som tall og [1] er selve noden
    
        for var, child_node in node.children.items():
            child_node.parent_node = node
            indices = [i for i, x in enumerate(node.data) if x == var] #indeksene til variablene til i datasettet til forelder noden
            result = count_values(y[indices]) #sjekker hvor mange true false man har for variabelen
            if len(result) == 1: 
                child_node.data = next(iter(result.keys())) #Alle er like, så vi gir verdi 
                child_node.isLeaf = True
            
            elif (len(X) == 1): #De er fortsatt ikke clean, men da må man finne den med flest dominerende
                child_node.data = find_dominant_value(result)
                child_node.isLeaf = True
                
            if not child_node.isLeaf:
                node_X = np.delete(X, index, axis=0)
                create_tree(node_X[:,indices],y[indices], child_node, impurity_measure) #når jeg skriver return her, så fucker alt seg 

In [25]:
def learn(X,y,impurity_measure = 'entropy',  pruning=False): 
    X_pruning, y_pruning = [],[]
    if pruning: 
        X, X_pruning, y, y_pruning =  get_pruning_set(X,y)
        
   
    root = Node()

    create_tree(X.T,y, root,impurity_measure)#Gir inn X.T Slik at hver av kategoriene kommer på en linje 
    if pruning: 
        while pruning: 
            pruning = False 
            pruning_pred = predict(X_pruning,root) #Vanlig predikt 
            pruning_accuracy = accuracy(pruning_pred, y_pruning)

            for children in root.children.values():
                leaf_node  = find_leaf(children)
                change_made = prune(leaf_node, pruning_accuracy,X_pruning,y_pruning, root)
                if (not pruning) and change_made: pruning = change_made
            
    return root

In [26]:
def get_pruning_set(X,y, size=0.2): 
    #lage en numpy funksjon som tar 
    random_selection = np.random.randint(len(y), size=int(len(y)*size))
    X_pruning, y_pruning = X[random_selection],  y[random_selection]
    X_train, y_train = np.delete(X, (random_selection), axis=0), np.delete(y, (random_selection), axis=0)
    return X_train, X_pruning, y_train, y_pruning

In [27]:
def find_leaf(node):
    if node.isLeaf: 
        return (node)
    else: 
        for key, child in node.children.items():
            #if not child.isLeaf:print(key, child.children.keys()) 
            return find_leaf(child)

In [28]:
#find the variabel 0 #sunny in grandparent node, so we can change to another node
#parent_variable_in_grand_parent
def find_parent_variabel(parent_node,grand_parent_node):
    for key, node in grand_parent_node.children.items(): 
        if node == parent_node: return key

In [29]:
def prune(leaf_node, pruning_accuracy, X_pruning, y_pruning, tree): 
    prun_acc = pruning_accuracy
    changes = False 
    parent = leaf_node.parent_node
    
    #Hvor mange utfall foreldernoden har.
    child_values_to_check = []
    child_values_to_check.append(bool(leaf_node.data))
    if(len(parent.children) >1): 
        child_values_to_check.append(not leaf_node.data)
    
    for i in child_values_to_check: 
        grand_parent = parent.parent_node

        dummy_node = Node()
        dummy_node.category = parent.category
        dummy_node.isLeaf = True
        dummy_node.data = i
        dummy_node.parent_node = grand_parent
        
        if not (grand_parent == None ): #Så lengde det ikke er root 
            parent_variable_in_grand_parent = find_parent_variabel(parent, grand_parent)
            grand_parent.children[parent_variable_in_grand_parent] = dummy_node
            
        #predict
        pred = predict(X_pruning, tree)
        pred_acc = accuracy(pred, y_pruning)
    
        if (prun_acc < pred_acc): 
            prun_acc = pred_acc
            parent = dummy_node
            changes = True
        elif not (grand_parent == None ): 
            grand_parent.children[parent_variable_in_grand_parent] = parent
    return changes 

In [31]:
tree = learn(X_train,y_train,pruning=True)

In [32]:
def print_tree(node):
    if node.isLeaf: 
        print("-----Leaf_value------")
        print(node.data)
        print()
    else: 
        for key, child in node.children.items():
            if not child.isLeaf:print(key, child.children.keys())
            print_tree(child)

In [33]:
print("Root: "+ str(tree.children.keys()))
print()
print_tree(tree)

Root: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])

-----Leaf_value------
0

-----Leaf_value------
1

-----Leaf_value------
1

3 dict_keys([1, 0, 4, 5, 3, 7, 6, 8])
-----Leaf_value------
1

-----Leaf_value------
1

4 dict_keys([5, 6, 3, 4, 1])
-----Leaf_value------
1

6 dict_keys([0, 2, 8, 1])
-----Leaf_value------
1

-----Leaf_value------
0

-----Leaf_value------
1

-----Leaf_value------
0

3 dict_keys([0, 1])
-----Leaf_value------
0

-----Leaf_value------
1

-----Leaf_value------
1

-----Leaf_value------
1

-----Leaf_value------
0

-----Leaf_value------
1

-----Leaf_value------
1

-----Leaf_value------
1

-----Leaf_value------
1

-----Leaf_value------
0

-----Leaf_value------
0

-----Leaf_value------
0

-----Leaf_value------
0

-----Leaf_value------
0



In [34]:
y_ = predict(X_test,tree)

In [35]:
accuracy(y_test,y_)

1.0